### Goal: create feature table that includes
- abnormal lab values
- min max mean of select lab values
- min max mean of select chart events
- sum of urine output
- demographic data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import statistics
import matplotlib.pyplot as plt

In [2]:
labevents_subset = pd.read_csv('./subsets_tables_collection/labevents_subset.csv')
labevents_subset['HADM_ID'] = labevents_subset['HADM_ID'].astype(np.int64)
labevents_subset['CHARTTIME'] = pd.to_datetime(labevents_subset['CHARTTIME'])
labevents_subset['ROW_ID'] = 'lab' + labevents_subset['ROW_ID'].astype(str)


chartevents_subset = pd.read_csv('./subsets_tables_collection/chartevents_subset.csv')
chartevents_subset['HADM_ID'] = chartevents_subset['HADM_ID'].astype(np.int64)
chartevents_subset['CHARTTIME'] = pd.to_datetime(chartevents_subset['CHARTTIME'])
chartevents_subset['ROW_ID'] = 'chart' + chartevents_subset['ROW_ID'].astype(str)

outputevents_subset = pd.read_csv('./subsets_tables_collection/outputevents_subset.csv')
outputevents_subset['HADM_ID'] = outputevents_subset['HADM_ID'].astype(np.int64)
outputevents_subset['CHARTTIME'] = pd.to_datetime(outputevents_subset['CHARTTIME'])
outputevents_subset['ROW_ID'] = 'output' + outputevents_subset['ROW_ID'].astype(str)

lab_times = labevents_subset[['ROW_ID','CHARTTIME','HADM_ID']]
chart_times = chartevents_subset[['ROW_ID','CHARTTIME','HADM_ID']]
out_times = outputevents_subset[['ROW_ID','CHARTTIME','HADM_ID']]
lab_chart_out_times = pd.concat([lab_times, chart_times, out_times], sort=False)
grouped = lab_chart_out_times.groupby(['HADM_ID', pd.Grouper(freq='48H', key='CHARTTIME')])
count = 0
past_hadm = ''
def counter(x):
    global count, past_hadm
    curr_hadm = x.iloc[0]
    if past_hadm != curr_hadm:
        count = 0
        past_hadm = curr_hadm
    y = count
    count += 1
    return str(curr_hadm) + "_" + str(count)
lab_chart_out_times['HADM_datebin_num'] = grouped['HADM_ID'].transform(counter)

/Users/chriszhou/anaconda/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# labevents_subset_binned = labevents_subset.merge(lab_chart_times, on=['ROW_ID', 'HADM_ID', 'CHARTTIME'])
# chartevents_subset_binned = chartevents_subset.merge(lab_chart_times, on=['ROW_ID', 'HADM_ID', 'CHARTTIME'])
labevents_subset_binned = labevents_subset.merge(lab_chart_out_times, on='ROW_ID')
chartevents_subset_binned = chartevents_subset.merge(lab_chart_out_times, on='ROW_ID')
outputevents_subset_binned = outputevents_subset.merge(lab_chart_out_times, on='ROW_ID')

In [4]:
import sqlite3 # library for working with sqlite database
conn = sqlite3.connect("./data/MIMIC.db") # Create a connection to the on-disk database
labitems = pd.read_sql("SELECT * FROM d_labitems", conn)
cols_to_use = ['ITEMID']
labevents_subset_binned = labevents_subset_binned.merge(labitems, on=cols_to_use)
labevents_subset_binned = labevents_subset_binned.drop(columns=['ROW_ID_y', 'index_y'])

### Make features out of abnormal lab values

In [5]:
labevents_subset_abnormal = labevents_subset_binned[labevents_subset_binned['FLAG'] == 'abnormal']
labevents_subset_normal = labevents_subset_binned[labevents_subset_binned['FLAG'] != 'abnormal']

In [6]:
# dropping lab categories detected abnormal less than 5 times
num_abnormal_labels = labevents_subset_abnormal.LABEL.value_counts()
abnormal_labels = num_abnormal_labels[num_abnormal_labels > 5].index
abnormal_labels

Index(['Glucose', 'Hematocrit', 'Hemoglobin', 'Red Blood Cells',
       'Urea Nitrogen', 'Creatinine', 'PT', 'RDW', 'Calcium, Total',
       'Chloride', 'pO2', 'PTT', 'Bicarbonate', 'Phosphate', 'MCH', 'Sodium',
       'Platelet Count', 'pCO2', 'White Blood Cells', 'INR(PT)', 'pH', 'MCHC',
       'MCV', 'Lactate', 'Troponin T', 'Free Calcium', 'Creatine Kinase (CK)',
       'Calculated Total CO2', 'Lymphocytes', 'Neutrophils', 'Potassium',
       'Albumin', 'Anion Gap', 'Magnesium', 'Bilirubin, Total',
       'Alkaline Phosphatase', 'Osmolality, Measured',
       'Asparate Aminotransferase (AST)', 'Creatine Kinase, MB Isoenzyme',
       'Lactate Dehydrogenase (LD)', 'Potassium, Whole Blood', 'RBC',
       'Alanine Aminotransferase (ALT)', 'CK-MB Index', 'Vancomycin',
       'Fibrinogen, Functional', 'Hyaline Casts', 'Sodium, Whole Blood',
       'Cortisol', 'WBC', 'Bands', 'Chloride, Whole Blood', 'Eosinophils',
       'Iron', 'Bilirubin, Direct', 'Gentamicin',
       'Calculated Bicar

In [7]:
labevents_subset_abnormal = labevents_subset_abnormal[labevents_subset_abnormal['LABEL'].isin(abnormal_labels)]

In [8]:
labevents_subset_abnormal = pd.get_dummies(labevents_subset_abnormal, prefix='abnormal_lab', columns=['LABEL'])

In [9]:
labevents_subset_features = pd.concat([labevents_subset_abnormal, labevents_subset_normal], sort=False)

In [10]:
labevents_subset_features = labevents_subset_features.drop('LABEL', axis=1)

In [11]:
filter_col = [col for col in labevents_subset_features if col.startswith('abnormal')]
filter_col.append('HADM_datebin_num')
labevents_subset_abnormal_features = labevents_subset_features[filter_col].groupby('HADM_datebin_num').sum()

### Extract numeric features out of chartevents

In [12]:
# What ITEMID from chartevents do we care about?
chart_ids_we_care = {
    'heart_rate': [211, 220045],
    'oxygen': [646, 220277, 834],
    'respiratory rate': [618,220210,3603,224689],
    'systolic blood pressure': [51, 455, 220179, 220050, 3313, 225309],
    'diastolic blood pressure': [8368, 8441, 220180, 220051, 8502, 225310],
    'mean blood pressure': [52, 456, 220181, 220052, 3312, 225312],
    'Glascow': [184, 723, 454, 220739, 223900, 223901],
    'temp F': [678, 223761, 679, ]
}
outputevents_ids_we_care = {
    'urine': [40055, 226559, 43175, 40069, 40094, 40065, 40061, 40715, 226627, 40473]
}
lab_ids_we_care = {
    'Hematocrit': [51221],
    'potassium': [50971],
    'sodium': [50983],
    'creatinine': [50912],
    'chloride': [50902],
    'platelets': [51265],
    'white blood cell': [51301],
    'hemoglobin': [51222],
    'glucose': [50931],
    'RBC count': [51279]
}

In [13]:
l = list(chart_ids_we_care.values())
flat_list = [item for sublist in l for item in sublist]
chartevents_subset_selected = chartevents_subset_binned[chartevents_subset_binned['ITEMID'].isin(flat_list)]
itemid_to_var_df = pd.read_csv('./resources/itemid_to_variable_map.csv')
chartevents_subset_selected = chartevents_subset_selected.merge(itemid_to_var_df, on='ITEMID')
cols_to_keep = ['SUBJECT_ID', 'HADM_ID_x', 'ITEMID', 'CHARTTIME_x', 'VALUE', 'VALUENUM', 'VALUEUOM', 'LEVEL2', 'HADM_datebin_num']
chartevents_subset_selected = chartevents_subset_selected[cols_to_keep]

In [14]:
chartevents_subset_selected = chartevents_subset_selected[chartevents_subset_selected['VALUENUM'].notnull()]

In [15]:
chartevents_features_max = chartevents_subset_selected.groupby(['HADM_datebin_num','LEVEL2'])['VALUENUM'].max().unstack()
chartevents_features_min = chartevents_subset_selected.groupby(['HADM_datebin_num','LEVEL2'])['VALUENUM'].min().unstack()
chartevents_features_mean = chartevents_subset_selected.groupby(['HADM_datebin_num','LEVEL2'])['VALUENUM'].mean().unstack()

In [16]:
chartevents_features_max = chartevents_features_max.add_suffix('_max')
chartevents_features_min = chartevents_features_min.add_suffix('_min')
chartevents_features_mean = chartevents_features_mean.add_suffix('_mean')
chartevents_features = pd.concat([chartevents_features_max, chartevents_features_min, chartevents_features_mean], axis=1)
# chartevents_features.reset_index(inplace=True)
chartevents_features.fillna(chartevents_features.mean(), inplace=True)

### Extract numeric features from lab events

In [17]:
l = list(lab_ids_we_care.values())
flat_list = [item for sublist in l for item in sublist]
labevents_subset_selected = labevents_subset_binned[labevents_subset_binned['ITEMID'].isin(flat_list)]
labevents_subset_selected = labevents_subset_selected.merge(itemid_to_var_df, on='ITEMID')
cols_to_keep = ['SUBJECT_ID', 'HADM_ID_x', 'ITEMID', 'CHARTTIME_x', 'VALUE', 'VALUENUM', 'VALUEUOM', 'LEVEL2', 'HADM_datebin_num']
labevents_subset_selected = labevents_subset_selected[cols_to_keep]
labevents_subset_selected = labevents_subset_selected[labevents_subset_selected['VALUENUM'].notnull()]
labevents_features_max = labevents_subset_selected.groupby(['HADM_datebin_num','LEVEL2'])['VALUENUM'].max().unstack()
labevents_features_min = labevents_subset_selected.groupby(['HADM_datebin_num','LEVEL2'])['VALUENUM'].min().unstack()
labevents_features_mean = labevents_subset_selected.groupby(['HADM_datebin_num','LEVEL2'])['VALUENUM'].mean().unstack()
labevents_features_max = labevents_features_max.add_suffix('_max')
labevents_features_min = labevents_features_min.add_suffix('_min')
labevents_features_mean = labevents_features_mean.add_suffix('_mean')
labevents_features = pd.concat([labevents_features_max, labevents_features_min, labevents_features_mean], axis=1)
# labevents_features.reset_index(inplace=True)
labevents_features.fillna(labevents_features.mean(),inplace=True)

### Extract numeric features from outevents

In [18]:
l = list(outputevents_ids_we_care.values())
flat_list = [item for sublist in l for item in sublist]
outputevents_subset_selected = outputevents_subset_binned[outputevents_subset_binned['ITEMID'].isin(flat_list)]
outputevents_subset_selected = outputevents_subset_selected.merge(itemid_to_var_df, on='ITEMID')
cols_to_keep = ['SUBJECT_ID', 'HADM_ID_x', 'ITEMID', 'CHARTTIME_x', 'VALUE', 'VALUEUOM', 'LEVEL2', 'HADM_datebin_num']
outputevents_subset_selected = outputevents_subset_selected[cols_to_keep]
outputevents_subset_selected = outputevents_subset_selected[outputevents_subset_selected['VALUE'].notnull()]
outputevents_features_sum = outputevents_subset_selected.groupby(['HADM_datebin_num','LEVEL2'])['VALUE'].sum().unstack()

outputevents_features_sum = outputevents_features_sum.add_suffix('_sum')
# labevents_features.reset_index(inplace=True)
outputevents_features_sum.fillna(outputevents_features_sum.mean(),inplace=True)

In [19]:
outputevents_features_sum

LEVEL2,Urine output_sum
HADM_datebin_num,
102390_2,5390.0
102390_3,4775.0
102390_4,3350.0
105348_1,1870.0
105348_2,1600.0
116543_1,205.0
116543_2,95.0
117382_1,980.0
117382_2,2160.0


### Merge Everything

In [20]:
feature_table = labevents_features.merge(chartevents_features, left_index=True, right_index=True, how='outer')
feature_table = feature_table.merge(labevents_subset_abnormal_features, left_index=True, right_index=True, how='outer')
feature_table = feature_table.merge(outputevents_features_sum, left_index=True, right_index=True, how='outer')


### Add HADM_ID back into the feature table, also create bin_num to mark which bin it is for that patient. these will be useful for assigning death labels

In [21]:
feature_table['HADM_ID'] = feature_table.index
feature_table[['HADM_ID', 'bin_num']] = feature_table['HADM_ID'].str.split('_', expand=True)
feature_table['HADM_ID'] = pd.to_numeric(feature_table['HADM_ID'])
feature_table['bin_num'] = pd.to_numeric(feature_table['bin_num'])

### Obtain number of diagnoses for each patient

In [22]:
diagnoses_subset = pd.read_csv('./subsets_tables_collection/diagnoses_icd_subset.csv')

In [23]:
diagnoses_count = diagnoses_subset.groupby('HADM_ID')['ICD9_CODE'].count()

### Obtain number of medications for each patient

In [24]:
prescriptions_subset = pd.read_csv('./subsets_tables_collection/prescriptions_subset.csv')
prescriptions_count = prescriptions_subset.groupby(['HADM_ID', 'DRUG_TYPE'])['DRUG'].count().unstack()
prescriptions_count.fillna(0, inplace=True)
prescriptions_count = prescriptions_count.add_suffix('_drug')

Merge diagnoses and medications

In [25]:
prescriptions_count.merge(diagnoses_count, left_index=True, right_index=True, how='outer')
prescriptions_count = prescriptions_count.add_suffix('_count')

### Incorporate demographics info

In [26]:
admissions_subset = pd.read_csv('./subsets_tables_collection/admissions_subset.csv')
patients = pd.read_sql("""SELECT subject_id, gender
                            FROM patients
                            """, conn)
patients.replace(['M','F'], [0,1], inplace=True)
patients.rename(columns={'GENDER': 'is_female'}, inplace=True)
admissions_with_gender = admissions_subset.merge(patients,on='SUBJECT_ID')
admissions_with_gender = admissions_with_gender[['HADM_ID', 'INSURANCE', 'ETHNICITY', 'is_female']]
admissions_features = pd.get_dummies(admissions_with_gender, prefix=['insurance','ethnicity'], columns=['INSURANCE', 'ETHNICITY'])
patient_features = admissions_features.merge(prescriptions_count, on='HADM_ID')

In [27]:
feature_table = feature_table.merge(patient_features, on='HADM_ID')

In [28]:
feature_table.dtypes

Chloride_max                            float64
Creatinine_max                          float64
Glucose_max                             float64
Hematocrit_max                          float64
Hemoglobin_max                          float64
Platelets_max                           float64
Potassium_max                           float64
Red blood cell count_max                float64
Sodium_max                              float64
White blood cell count_max              float64
Chloride_min                            float64
Creatinine_min                          float64
Glucose_min                             float64
Hematocrit_min                          float64
Hemoglobin_min                          float64
Platelets_min                           float64
Potassium_min                           float64
Red blood cell count_min                float64
Sodium_min                              float64
White blood cell count_min              float64
Chloride_mean                           